In [1]:
%matplotlib inline
# this program is going to estimate parameters from simulated datasets that originate from an OU process
# with noise added.  The parameters of the simulation and the length of the simulation are set through
# arguments
import langevin
import pandas as pd
import numpy as np
import argparse
import pymc3 as pm
import theano.tensor as tt
from scipy.stats import pearsonr
from scipy.optimize import root



In [2]:
class Ornstein_Uhlenbeck(pm.Continuous):
    """
    Ornstein-Uhlenbeck Process
    Parameters
    ----------
    B : tensor
        B > 0, B = exp(-(D/A)*delta_t)
    A : tensor
        A > 0, amplitude of fluctuation <x**2>=A
    delta_t: scalar
        delta_t > 0, time step
    """
    
    def __init__(self, A=None, B=None,
                 *args, **kwargs):
        super(Ornstein_Uhlenbeck, self).__init__(*args, **kwargs)
        self.A = A
        self.B = B
        self.mean = 0.
    
    def logp(self, x):
        A = self.A
        B = self.B
        
        x_im1 = x[:-1]
        x_i = x[1:]
        
        ou_like = pm.Normal.dist(mu=x_im1*B, tau=1.0/A/(1-B**2)).logp(x_i)
        return pm.Normal.dist(mu=0.0,tau=1.0/A).logp(x[0]) + tt.sum(ou_like)

In [9]:
def calc_fundstats(x):
    return x[0]**2+x[-1]**2,np.sum(x[1:-1]**2),np.sum(x[0:-1]*x[1:])


In [3]:
def correlated_ts(c,delta_t = 0.1,N=1000):
    # parameters for coupled oscillator
    K,D = 1.0,1.0
    data1 = langevin.time_series(A=1/K, D=D, delta_t=delta_t, N=N)
    data2 = langevin.time_series(A=1/(K+np.abs(c)), D=D, delta_t=delta_t, N=N)
    x1 = (data1 + data2)/2
    if c>0:
        x2 = (data1 - data2)/2
    else:
        x2 = (data2-data1)/2

    return x1,x2

In [14]:
#parameters
a_bound=5
rho=0.9
N=1000
M=100
delta_t = 0.3

In [15]:
results = None
for i in range(M):
    coupling = 2*np.abs(rho)/(1-np.abs(rho))*np.sign(rho)
    x1,x2 = correlated_ts(coupling,delta_t=delta_t,N=N)
    prho = pearsonr(x1,x2)[0]
    print("pearson: ",prho)

    para = calc_fundstats(x1+x2) + calc_fundstats(x1-x2)
    y1 = x1 + x2
    y2 = x1 - x2
    with pm.Model() as model:
        A1 = pm.Uniform('A1', lower=0, upper=a_bound)
        A2 = pm.Uniform('A2', lower=0, upper=a_bound)
        D = pm.Uniform('D',lower=0,upper=5)
            
        B1 = pm.Deterministic('B1',pm.math.exp(-delta_t * D / A1))
        B2 = pm.Deterministic('B2',pm.math.exp(-delta_t * D / A2))
                                
        path1 = Ornstein_Uhlenbeck('path1',A=A1, B=B1,shape=len(y1),observed=y1)
        path2 = Ornstein_Uhlenbeck('path2',A=A2, B=B2,shape=len(y2),observed=y2)
                                
        trace = pm.sample(10000,tune=2000)

    A1_trace = trace['A1']
    A2_trace = trace['A2']
    A1_mean = np.mean(A1_trace)
    A2_mean = np.mean(A2_trace)
    dA1 = np.std(A1_trace)
    dA2 = np.std(A2_trace)
    
    D_trace = trace['D']
    D_mean = np.mean(D_trace)
    dD = np.std(D_trace)

    dA1A2 = np.cov(A1_trace,A2_trace)[0,1]
    dA1D = np.cov(A1_trace,D_trace)[0,1]
    dA2D = np.cov(A2_trace,D_trace)[0,1]
    
    C_trace = (A1_trace-A2_trace)/A2_trace
    C_mean = np.mean(C_trace)
    dC = np.std(C_trace)

    print("predicted C: ",C_mean," +- ",dC)

    if results is None:
        results = [rho,
                   coupling,
                   prho,
                   C_mean,
                   dC,
                   A1_mean,
                   dA1,
                   A2_mean,
                   dA2,
                   D_mean,
                   dD,
                   dA1A2,
                   dA1D,
                   dA2D,
                   para[0],
                   para[1],
                   para[2],
                   para[3],
                   para[4],
                   para[5],
                   C_mean,
                   dC
                   ]
    else:
        results = np.vstack((results,[rho,
                   coupling,
                   prho,
                   C_mean,
                   dC,
                   A1_mean,
                   dA1,
                   A2_mean,
                   dA2,
                   D_mean,
                   dD,
                   dA1A2,
                   dA1D,
                   dA2D,
                   para[0],
                   para[1],
                   para[2],
                   para[3],
                   para[4],
                   para[5],
                   C_mean,
                   dC
                   ]))


pearson:  0.8972868301178325


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [D, A2, A1]
Sampling 4 chains, 0 divergences: 100%|██████████| 48000/48000 [00:17<00:00, 2807.17draws/s]


predicted C:  18.055542701966537  +-  1.8491499454120297
pearson:  0.8863937199690196


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [D, A2, A1]
Sampling 4 chains, 0 divergences: 100%|██████████| 48000/48000 [00:18<00:00, 2554.29draws/s]


predicted C:  15.877805030196056  +-  1.5974955512651399
pearson:  0.9007545458631666


...

predicted C:  18.744374720209603  +-  1.895776762514879
pearson:  0.8961278501175216


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [D, A2, A1]
Sampling 4 chains, 0 divergences: 100%|██████████| 48000/48000 [00:16<00:00, 2949.22draws/s]


predicted C:  17.512417474511377  +-  1.727418486870094
pearson:  0.9044282489034969


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [D, A2, A1]
Sampling 4 chains, 0 divergences: 100%|██████████| 48000/48000 [00:16<00:00, 2824.48draws/s]


predicted C:  19.348758879515696  +-  2.0944336532298715


In [16]:
results

array([[ 0.9       , 18.        ,  0.89728683, ..., -1.05231367,
        18.0555427 ,  1.84914995],
       [ 0.9       , 18.        ,  0.88639372, ...,  3.22401807,
        15.87780503,  1.59749555],
       [ 0.9       , 18.        ,  0.90075455, ..., -1.03085751,
        19.28885755,  1.99440336],
       ...,
       [ 0.9       , 18.        ,  0.90162166, ...,  3.16720913,
        18.74437472,  1.89577676],
       [ 0.9       , 18.        ,  0.89612785, ..., -1.36454011,
        17.51241747,  1.72741849],
       [ 0.9       , 18.        ,  0.90442825, ..., -2.23707317,
        19.34875888,  2.09443365]])

In [ ]:
results = DataFrame(rho = Float64[],
                    coupling = Float64[],
                    pearson = Float64[],
                    a1 = Float64[],
                    da1 = Float64[],
                    a2 = Float64[],
                    da2 = Float64[],
                    d = Float64[],
                    dd = Float64[],
                    da1da2 = Float64[],
                    da1dd = Float64[],
                    dasdd = Float64[],
                    a1ep = Float64[],
                    a1ss = Float64[],
                    a1c = Float64[],
                    a2ep = Float64[],
                    a2ss = Float64[],
                    a2c = Float64[],
                    c = Float64[],
                    dc = Float64[])


In [17]:
column_names = ["rho",
                "coupling",
                "pearson",
                "C",
                "dC",
                 "a1",
                "da1",
                "a2",
                "da2",
                "d",
                "dd",
                "da1da2",
                "da1dd",
                "dasdd",
                "a1ep",
                "a1ss",
                "a1c",
                "a2ep",
                "a2ss",
                "a2c",
                "c",
                "dc"]
df=pd.DataFrame(results,columns=column_names)
print(df)
df.to_csv('correlations1k09_PYMC.csv',index=False)

    rho  coupling   pearson          C        dC        a1       da1  \
0   0.9      18.0  0.897287  18.055543  1.849150  1.013134  0.087223   
1   0.9      18.0  0.886394  15.877805  1.597496  0.981562  0.081639   
2   0.9      18.0  0.900755  19.288858  1.994403  1.062511  0.092750   
3   0.9      18.0  0.880274  14.973885  1.496495  0.889274  0.073017   
4   0.9      18.0  0.897228  17.810677  1.845325  1.021282  0.088738   
..  ...       ...       ...        ...       ...       ...       ...   
95  0.9      18.0  0.900160  18.289409  1.780469  1.024044  0.082412   
96  0.9      18.0  0.903438  19.164847  1.923714  0.946645  0.079371   
97  0.9      18.0  0.901622  18.744375  1.895777  0.981797  0.082615   
98  0.9      18.0  0.896128  17.512417  1.727418  0.936731  0.076584   
99  0.9      18.0  0.904428  19.348759  2.094434  1.056331  0.097724   

          a2       da2         d  ...     da1dd         dasdd      a1ep  \
0   0.053275  0.002410  1.015557  ...  0.000145  2.551072e-0